In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00


In [2]:
! pip install category-encoders
import category_encoders
from category_encoders.target_encoder import TargetEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.7 MB/s eta 0:00:00


In [3]:
%%writefile heart_disease.py
import streamlit as st
from joblib import load
import numpy as np
from PIL import Image
import pandas as pd

pipe = load('/content/pipe.joblib')
model_1 = load('/content/model_1.joblib')
model_2 = load('/content/model_2.joblib')


st.set_page_config(page_title="Heart Disease Predictor", layout="wide")

def main_page():
    #st.title("# Heart Disease Predictor by Alba Puig")
    st.header('Heart Disease Predictor', divider='grey')
    st.subheader('by Alba Puig')
    st.markdown(""" Benvinguts a l'aplicació de predicció de malalties cardíaques. Aquesta eina ha estat dissenyada per a dos tipus d’usuaris:
    pacients i professionals mèdics. Com a pacient, podràs introduir les teves dades de salut a través d’una interfície amigable i intuïtiva, i obtenir
    una predicció basada en models científics avançats. Si ets un professional mèdic, tindràs accés a un model més detallat que et permetrà fer servir dades
    més específiques per a les teves prediccions. Ambdós models han estat entrenats amb dades reals i validats amb rigor científic.

    Per utilitzar la interfície, simplement segueix aquests passos:

    1. Selecciona el teu tipus d’usuari: A la pàgina d’inici, selecciona si ets un pacient o un professional mèdic. Això et portarà a un formulari específic per al teu tipus d’usuari. \n

    2. Introdueix les teves dades de salut: Utilitza els controls d’entrada per introduir les teves dades de salut. Aquests controls inclouen sliders, botons i caixes de selecció que et permeten introduir les teves dades de manera intuïtiva. \n

    3. Fes clic al botó “Finish”: Un cop hagis introduït totes les teves dades, fes clic al botó “Finish”. Això processarà les teves dades a través del nostre pipeline i es realitzarà una predicció utilitzant el model pertinent. \n

    4. Consulta la teva predicció: Finalment, podràs veure la predicció basada en les dades que has introduït. Recorda que aquesta predicció és només una eina i no substitueix el consell mèdic professional. \n

    Esperem que aquesta eina t’ajudi a entendre millor la teva salut cardiovascular. Si tens alguna pregunta o necessites ajuda, no dubtis a posar-te en contacte amb nosaltres. \n """)

    image = Image.open('/content/img.png')
    st.image(image)

def Patient(pipe,model_1):
    st.markdown("# Patient 👩‍💼👨‍💼")
    st.sidebar.markdown("""1. Introdueix les teves dades de salut utilitzant els controls d’entrada. \n """)
    st.sidebar.markdown("""2. Fes clic al botó “Finish” per processar les teves dades i obtenir una predicció. \n""")
    st.sidebar.markdown("""3. Consulta la teva predicció. Recorda, aquesta eina no substitueix el consell mèdic professional.""")

    age_ = st.slider('Age:', 0, 100)
    dataset_ = st.selectbox('Dataset:', ('Cleveland', 'Hungary', 'Switzerland','VA Long Beach'))
    sex_ = st.radio("Gender:", ['Male', 'Female'])
    cp_ = st.selectbox('Chest Pain:', ('Typical Angina', 'Asymptomatic', 'Non-Anginal ','Atypical Angina'))
    trestbps_ = st.slider('Resting blood pressure in mmHg:', 70, 220)
    chol_ = st.slider('Serum cholesterol in mg/dl:', 80, 700)
    fbs_ = st.radio( "Fasting blood sugar > 120 mg/dl:", ["True", "False"])
    restecg_ = float("NAN")
    thalch_ = st.slider('Maximum heart rate achieved:', 50, 220)
    exang_ = st.radio( "Exercise-induced angina:", ["True", "False"])
    oldpeak_ = float("NAN")
    slope_ = float("NAN")
    ca_ = float("NAN")
    thal_ = st.selectbox('Thalassemia (blood disorder):', ('Fixed defect ', 'Normal', 'Reversable defect'))

    if st.button('Finish'):
      st.header('Results:')
      input_data_m1 = [age_, sex_, dataset_, cp_, trestbps_, chol_, fbs_, restecg_, thalch_, exang_, oldpeak_, slope_, ca_, thal_]
      transformed_data_m1 = pipe.transform(pd.DataFrame([input_data_m1],columns=['age', 'sex', 'dataset', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalch', 'exang', 'oldpeak', 'slope', 'ca', 'thal']))
      probability_m1 = model_1.predict_proba(transformed_data_m1)
      threshold = 0.31

      #st.write(probability_m1)

      # Display the probabilities
      #st.write('Probabilities:')
      #st.write(f'Probability of class 0: {probability_m1[:, 0][0]}')
      #st.write(f'Probability of class 1: {probability_m1[:, 1][0]}')

      if(probability_m1[:, 1][0] > threshold):
        st.subheader("""High probability of heart disease. It is recommended that you consult with a medical professional for an accurate diagnosis.""")
      else:
        st.subheader("""No significant signs of heart disease are detected based on the provided data. However, remember that this is a supportive tool and does not replace a medical consultation.""")

def Doctor(pipe,model_2):
    st.markdown("# Doctor 👩‍⚕️👨‍⚕️")
    st.sidebar.markdown("""1. Introdueix les teves dades de salut utilitzant els controls d’entrada. \n """)
    st.sidebar.markdown("""2. Fes clic al botó “Finish” per processar les teves dades i obtenir una predicció. \n""")
    st.sidebar.markdown("""3. Consulta la teva predicció.""")

    age = st.slider('Age:', 0, 100)
    dataset = st.selectbox('Dataset:', ('Cleveland', 'Hungary', 'Switzerland','VA Long Beach'))
    sex = st.radio("Gender:", ['Male', 'Female'])
    cp = st.selectbox('Chest Pain:', ('Typical Angina', 'Asymptomatic', 'Non-Anginal ','Atypical Angina'))
    trestbps = st.slider('Resting blood pressure in mmHg:', 70, 220)
    chol = st.slider('Serum cholesterol in mg/dl:', 80, 700)
    fbs = st.radio( "Fasting blood sugar > 120 mg/dl:", ["True", "False"])
    restecg = st.selectbox('Resting electrocardiographic results:', ('Lv hypertrophy', 'Normal', 'St-t abnormality'))
    thalch = st.slider('Maximum heart rate achieved:', 50, 220)
    exang = st.radio( "Exercise-induced angina:", ["True", "False"])
    oldpeak = st.slider('ST depression induced by exercise relative to rest:', -2.6, 6.2)
    slope = st.selectbox('The slope of the peak exercise ST segment:', ('Downsloping ', 'Flat', 'Upsloping'))
    ca = st.radio( "Number of major vessels colored by fluoroscopy:", ["0", "1", "2", "3"])
    thal = st.selectbox('Thalassemia (blood disorder):', ('Fixed defect ', 'Normal', 'Reversable defect'))

    if st.button('Finish'):
       st.header('Results:')
       input_data_m2 = [age, sex, dataset, cp, trestbps, chol, fbs, restecg, thalch, exang, oldpeak, slope, ca, thal]
       transformed_data_m2 = pipe.transform(pd.DataFrame([input_data_m2],columns=['age', 'sex', 'dataset', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalch', 'exang', 'oldpeak', 'slope', 'ca', 'thal']))
       probability_m2 = model_2.predict(transformed_data_m2)

       if(probability_m2[0] == 0):
           st.subheader("""The patient shows no significant signs of heart disease based on the provided data.""")
       elif(probability_m2[0] == 1):
           st.subheader("""The patient shows signs indicative of Stage 1 heart disease. This is an early stage of the disease, and immediate medical consultation is advised for further diagnosis and treatment.""")
       elif(probability_m2[0] == 2):
           st.subheader("""The patient shows signs indicative of Stage 2 heart disease. This is a more advanced stage of the disease, and immediate medical consultation is advised for further diagnosis and treatment.""")
       elif(probability_m2[0] == 3):
           st.subheader("""The patient shows signs indicative of Stage 3 heart disease. This is a severe stage of the disease, and immediate medical consultation is advised for further diagnosis and treatment.""")
       else:
           st.subheader("""The patient shows signs indicative of Stage 4 heart disease. This is the most advanced stage of the disease, and immediate medical consultation is advised for further diagnosis and treatment.""")


page_names_to_funcs = {
    "Heart Disease Predictor": main_page,
    "Patient": lambda: Patient(pipe,model_1),
    "Doctor": lambda: Doctor(pipe,model_2),
}

selected_page = st.sidebar.selectbox("Select user type", page_names_to_funcs.keys())
page_names_to_funcs[selected_page]()

Writing heart_disease.py


In [4]:
!wget -q -O - ipv4.icanhazip.com

34.83.205.51


Copiar aquest codi i enganxar-lo al URL donat a continuació.

In [5]:
!streamlit run heart_disease.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.205.51:8501

npx: installed 22 in 10.819s
your url is: https://odd-shirts-drum.loca.lt
  Stopping...
^C
